In [1]:
# LIBRARIES #

import numpy as np

import SimpleITK

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot
from bokeh.layouts import row, gridplot

output_notebook()

Loading BokehJS ...

In [2]:
# INPUT PARAMETERS #

#m_filename = 'G:/Commun/PHYSICIENS/CQ/CQ_interne_BV/Cyberknife M6/CQ mensuel/2019/2019 02 19/champs lumineux - irradie.tif'
m_filename = 'Film_3_2019-02-13004.tif'

m_coefA = 1774.1   #x3
m_coefB = -3907.8  #x2
m_coefC = 3276.7   #x
m_coefD = -935.12

m_dosemax = 1100.0 #cGy


In [3]:
# READS THE IMAGE AND CONVERTS IT TO DOSE #

def readAndConvertImgToDose(filename, coefs):
    # reads the image using simpleITK:
    img = SimpleITK.ReadImage(filename)
    sizex = img.GetWidth()
    sizey = img.GetHeight()
    array = SimpleITK.GetArrayFromImage(img)
    
    # replaces every 65535 value in array with 65534 to avoid division by zero:
    array[array==65535]=65534
    
    # converts in optical density
    dor = -np.log10(array[:,:,0]/65535.0)
    dob = -np.log10(array[:,:,2]/65535.0)
    
    # red channel over blue channel:
    rsb = dor/dob
#    rsb[rsb>1.3] = 1.3

    # converting in dose:
    dose = coefs[0]*rsb**3 + coefs[1]*rsb**2 + coefs[2]*rsb + coefs[3]
    dose[dose>m_dosemax] = m_dosemax
    
    return dose, sizex, sizey

In [4]:
# READS THE IMAGE AND CONVERTS IT TO RGBA IMG #

def readAndConvertImgToRGBA(filename, coefs):
    # reads the image using simpleITK:
    img = SimpleITK.ReadImage(filename)
    sizex = img.GetWidth()
    sizey = img.GetHeight()
    array = SimpleITK.GetArrayFromImage(img)
    
    # creates a new rgba img and copy the tiff values in it
    img = np.empty((sizey,sizex), dtype=np.uint32)
    view = img.view(dtype=np.uint8).reshape((sizey, sizex, 4))
    view[:,:,0] = array[:,:,0]/65535.0*255.0
    view[:,:,1] = array[:,:,1]/65535.0*255.0
    view[:,:,2] = array[:,:,2]/65535.0*255.0
    view[:,:,3] = 255

    return img, sizex, sizey

In [7]:
# PLOTS IMAGES #

coefs = [m_coefA, m_coefB, m_coefC, m_coefD]

doseimg, sizex, sizey = readAndConvertImgToDose(m_filename, coefs)
maxdose = int(np.amax(doseimg))
print('Image dimension (in pixels):', sizex, 'x', sizey)
print('Dose max:', maxdose, 'cGy')

rgbaimg, sizex, sizey = readAndConvertImgToRGBA(m_filename, coefs)

color_mapper = LinearColorMapper(palette="Viridis256", low=0, high=maxdose)

color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0),
                     title='Dose cGy')

p1 = figure(plot_width=int(450*1.1), plot_height=int(450*sizey/sizex), x_range=(0,sizex), y_range=(0,sizey), 
            title="Dose image", toolbar_location="above")

p1.image(image=[doseimg], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper)

p1.add_layout(color_bar, 'right')

p2 = figure(plot_width=int(450*0.95), plot_height=int(450*0.95*sizey/sizex), x_range=(0,sizex), y_range=(0,sizey), 
            title="Gafchromic image", toolbar_location="above")

p2.image_rgba(image=[rgbaimg], x=[0], y=[0], dw=[sizex], dh=[sizey])

show(row(p1,p2))

Image dimension (in pixels): 589 x 144
Dose max: 1100 cGy


In [24]:
# PLOTS IMAGE AND PROFILES #


# Line profiles to draw:
xLinePos = 90
yLinePos = 300
ImgPlotWidth = 600

# Reads the image and convert it to dose:
coefs = [m_coefA, m_coefB, m_coefC, m_coefD]

doseimg, sizex, sizey = readAndConvertImgToDose(m_filename, coefs)
maxdose = int(np.amax(doseimg))

print('Image dimension (in pixels):', sizex, 'x', sizey)
print('Dose max:', maxdose, 'cGy')


# Displays the dose image (p1):
color_mapper = LinearColorMapper(palette="Viridis256", low=0, high=maxdose)

color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0),
                     title='Dose cGy')

p1 = figure(plot_width=int(ImgPlotWidth*1.1), plot_height=int(ImgPlotWidth*sizey/sizex), x_range=(0,sizex), y_range=(0,sizey), 
            title="Dose image", toolbar_location="above")

p1.image(image=[doseimg], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper)
p1.line([0,int(sizex)], [xLinePos,xLinePos], line_width=2, line_color=(255, 255, 255, 0.3))
p1.line([yLinePos, yLinePos], [0,int(sizey)], line_width=2, line_color=(255, 255, 255, 0.3))

p1.add_layout(color_bar, 'right')


# Displays the profiles (p2):
maxdose_x = np.amax(doseimg[xLinePos,:])
p2 = figure(plot_width=500, plot_height=300, x_range=(0,sizex), y_range=(0,maxdose_x), 
            title="Horizontal dose profile (y = " + str(xLinePos) + "pix)", toolbar_location="above")

p2_xvalues = np.arange(0, sizex, 1)
p2_yvalues = doseimg[xLinePos,:]

p2.line(x=p2_xvalues, y=p2_yvalues, line_color="#f46d43", line_width=3, line_alpha=1.0)


# Displays the profiles (p3):
maxdose_x = np.amax(doseimg[:,yLinePos])
p3 = figure(plot_width=500, plot_height=300, x_range=(0,sizey), y_range=(0,maxdose_x), 
            title="Vertical dose profile (y = " + str(yLinePos) + "pix)", toolbar_location="above")

p3_xvalues = np.arange(0, sizey, 1)
p3_yvalues = doseimg[:,yLinePos]

p3.line(x=p3_xvalues, y=p3_yvalues, line_color="#f46d43", line_width=3, line_alpha=1.0)


# plotting inline:
grid = gridplot([[p1, None], [p2,p3]])

show(grid)

Image dimension (in pixels): 589 x 144
Dose max: 1100 cGy
